In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [246]:
%autoreload
%matplotlib inline

import numpy as np
import pickle

import main


import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc_file('matplotlibrc')

import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.spectools as spectools

from darkhistory.spec.spectrum import Spectrum

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec

from darkhistory.electrons import positronium

from scipy.interpolate import interp1d


In [117]:
test = positronium.weighted_photon_spec(photeng)
print(test.toteng()/phys.me)

2.000000000081069


# Marching Up Algorithm

In [3]:
ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf = main.load_ics_data()


********* Thomson regime scattered photon spectrum *********
Initializing...
Computing spectra by an expansion in beta...
----> Computation by expansion in beta complete!
Computing spectra by analytic series...
*** Computing series 1/12...
*** Computing series 2/12...
*** Computing series 3/12...
*** Computing series 4/12...
*** Computing series 5/12...
*** Computing series 6/12...
*** Computing series 7/12...
*** Computing series 8/12...
*** Computing series 9/12...
*** Computing series 10/12...
*** Computing series 11/12...
*** Computing series 12/12...
----> Computation by analytic series complete!
Spectrum computed!
********* Relativistic regime scattered photon spectrum *********
Initializing...
Computing series 1/4...
Computing series 2/4...
Computing series 3/4...
Computing series 4/4...
Relativistic Computation Complete!
********* Thomson regime energy loss spectrum *********
Computing nonrelativistic energy loss spectrum...
Computing energy loss spectrum by beta expansion...
C

In [252]:
pickle.dump(ics_thomson_ref_tf, open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/ics/ics_thomson_ref_tf.raw", "wb"))
pickle.dump(ics_rel_ref_tf, open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/ics/ics_rel_ref_tf.raw", "wb"))
pickle.dump(engloss_ref_tf, open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/tflists/ics/engloss_ref_tf.raw", "wb"))

In [253]:
def get_elec_cooling_tf_fast(
    raw_nonrel_tf, raw_rel_tf, raw_engloss_tf,
    eleceng, photeng, rs, xe, xHe=0
):

    """Returns transfer function for complete electron cooling through ICS and atomic processes.

    Parameters
    ----------
    nonrel_tf : TransFuncAtRedshift
        Raw nonrelativistic primary electron ICS transfer function.
    rel_tf : string
        Raw relativistic primary electron ICS transfer function.
    engloss_tf_filename : string
        Raw primary electron ICS energy loss transfer function.
    eleceng : ndarray
        The electron *kinetic* energy abscissa.
    photeng : ndarray
        The photon energy abscissa.
    rs : float
        The redshift.
    xe : float
        Free electron fraction. 
    xHe : float, optional
        Singly-ionized helium fraction, nHe+/nH. Set to nHe/nH*xe if None.
    
    Returns
    -------

    tuple of TransFuncAtRedshift
        Transfer functions for photons and low energy electrons.

    Note
    ----
    The raw transfer functions should be generated when the code package is first installed. The transfer function corresponds to the fully resolved
    photon spectrum after scattering by one electron.

    This version of the code works faster, but dispenses with energy conservation checks and several other safeguards. Use only with default abscissa, or when get_ics_cooling_tf works.

    """
    
    if xHe is None:
        xHe = xe*phys.nHe/phys.nH
        
    # v/c of electrons, important subsequently.
    beta_ele = np.sqrt(1 - 1/(1 + eleceng/phys.me)**2)
        
    #####################################
    # Inverse Compton
    #####################################

    T = phys.TCMB(rs)

    # Photon transfer function for single primary electron single scattering.
    # This is dN/(dE dt), dt = 1 s.
    phot_ICS_tf = ics_spec(
        eleceng, photeng, T, nonrel_tf = raw_nonrel_tf, rel_tf = raw_rel_tf
    )

    # Downcasting speeds up np.dot
    phot_ICS_tf._grid_vals = phot_ICS_tf.grid_vals.astype('float64')

    # Energy loss transfer function for single primary electron
    # single scattering. This is dN/(dE dt), dt = 1 s.
    engloss_ICS_tf = engloss_spec(
        eleceng, photeng, T, nonrel_tf = raw_engloss_tf, rel_tf = raw_rel_tf
    )

    # Downcasting speeds up np.dot
    engloss_ICS_tf._grid_vals = engloss_ICS_tf.grid_vals.astype('float64')

    # Switch the spectra type here to type 'N'.
    if phot_ICS_tf.spec_type == 'dNdE':
        phot_ICS_tf.switch_spec_type()
    if engloss_ICS_tf.spec_type == 'dNdE':
        engloss_ICS_tf.switch_spec_type()


    # Define some useful lengths.
    N_eleceng = eleceng.size
    N_photeng = photeng.size

    # Create the secondary electron transfer functions.

    # ICS transfer function.
    elec_ICS_tf = tf.TransFuncAtRedshift(
        np.zeros((N_eleceng, N_eleceng)), in_eng = eleceng,
        rs = rs*np.ones_like(eleceng), eng = eleceng,
        dlnz = -1, spec_type = 'N'
    )

    elec_ICS_tf._grid_vals = spectools.engloss_rebin_fast(
        eleceng, photeng, engloss_ICS_tf.grid_vals, eleceng
    )
    
    # Total upscattered photon energy.
    cont_loss_ICS_vec = np.zeros_like(eleceng)
    # Deposited energy, enforces energy conservation.
    deposited_ICS_vec = np.zeros_like(eleceng)
    
    
    #####################
    # Excitation  
    #####################
    
    # Construct the rate matrices first. Secondary electron spectrum is an electron at in_eng - excitation energy, 
    # with a per second rate given by n*sigma*c.
    

    rate_matrix_exc_HI = np.diag(
        (1 - xe)*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HI') * beta_ele * phys.c
    )
    
    rate_matrix_exc_HeI = np.diag(
        (phys.nHe/phys.nH - xHe)*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HeI') * beta_ele * phys.c
    )
    
    rate_matrix_exc_HeII = np.diag(
        xHe*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HeII') * beta_ele * phys.c
    )

    # Construct the TransFuncAtRedshift objects.
    # Electrons scatter from in_eng to in_eng - excitation energy.
    # Remove all of the columns (eng) that have energies below the excitation energy, 
    elec_exc_HI_tf = tf.TransFuncAtRedshift(
        np.squeeze(rate_matrix_exc_HI[:, np.where(eleceng > phys.lya_eng)]), 
        in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng[eleceng > phys.lya_eng] - phys.lya_eng,
        dlnz = -1, spec_type = 'N'
    )
    elec_exc_HeI_tf = tf.TransFuncAtRedshift(
        np.squeeze(rate_matrix_exc_HeI[:, np.where(eleceng > phys.He_exc_eng)]), 
        in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng[eleceng > phys.He_exc_eng] - phys.He_exc_eng,
        dlnz = -1, spec_type = 'N'
    )
    elec_exc_HeII_tf = tf.TransFuncAtRedshift(
        np.squeeze(rate_matrix_exc_HeII[:, np.where(eleceng > 4*phys.lya_eng)]), 
        in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng[eleceng > 4*phys.lya_eng] - 4*phys.lya_eng,
        dlnz = -1, spec_type = 'N'
    )
    
    # Rebin these transfer functions back to eleceng.
    elec_exc_HI_tf.rebin(eleceng)
    elec_exc_HeI_tf.rebin(eleceng)
    elec_exc_HeII_tf.rebin(eleceng)
   
    # Deposited energy for excitation.
    deposited_exc_vec = np.zeros_like(eleceng)
    
    #####################
    # Ionization  
    #####################
    
    # Construct the rate vector first. Secondary electron spectrum is an electron at in_eng - excitation energy, 
    # with a per second rate given by n*sigma*c.
    rate_vec_ion_HI = (
        (1 - xe)*phys.nH*rs**3 * phys.coll_ion_xsec(eleceng, species='HI') * beta_ele * phys.c
    )
    
    rate_vec_ion_HeI = (
        (phys.nHe/phys.nH - xHe)*phys.nH*rs**3 * phys.coll_ion_xsec(eleceng, species='HeI') * beta_ele * phys.c
    )
    
    rate_vec_ion_HeII = (
        xHe*phys.nH*rs**3 * phys.coll_ion_xsec(eleceng, species='HeII') * beta_ele * phys.c
    )
    
    # Construct the spectra. 
    elec_spec_ion_HI = np.array(
        [rate*phys.coll_ion_sec_elec_spec(in_eng, eleceng, species='HI') for rate,in_eng in zip(rate_vec_ion_HI,eleceng)]
    )
    elec_spec_ion_HeI = np.array(
        [rate*phys.coll_ion_sec_elec_spec(in_eng, eleceng, species='HeI') for rate,in_eng in zip(rate_vec_ion_HeI,eleceng)]
    )
    elec_spec_ion_HeII = np.array(
        [rate*phys.coll_ion_sec_elec_spec(in_eng, eleceng, species='HeII') for rate,in_eng in zip(rate_vec_ion_HeII,eleceng)]
    )
    
#     for i in np.arange(500):
# #         print(np.dot(elec_spec_ion_HI[i], eleceng) + rate_vec_ion_HI[i]*phys.rydberg - eleceng[i]*rate_vec_ion_HI[i])
#         print('Injection energy: ', eleceng[i])
#         print(np.dot(elec_spec_ion_HeI[i], eleceng))
#         print(rate_vec_ion_HeI[i]*phys.He_ion_eng)
#         print(eleceng[i]*rate_vec_ion_HeI[i])
#         print(np.dot(elec_spec_ion_HeI[i], eleceng) + rate_vec_ion_HeI[i]*phys.He_ion_eng - eleceng[i]*rate_vec_ion_HeI[i])
    
    
    
    # Construct the TransFuncAtRedshift objects.
    # Electrons scatter from in_eng to in_eng - excitation energy.
    # Remove all of the columns (eng) that have energies below the excitation energy, 
    elec_ion_HI_tf = tf.TransFuncAtRedshift(
        elec_spec_ion_HI, in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng, dlnz = -1, spec_type = 'N'
    )
    elec_ion_HeI_tf = tf.TransFuncAtRedshift(
        elec_spec_ion_HeI, in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng, dlnz = -1, spec_type = 'N'
    )
    elec_ion_HeII_tf = tf.TransFuncAtRedshift(
        elec_spec_ion_HeII, in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng, dlnz = -1, spec_type = 'N'
    )
    
    # Deposited energy for ionization.
    deposited_ion_vec = np.zeros_like(eleceng)
    
    #############################################
    # Heating
    #############################################
    
    dE_heat_dt = phys.elec_heating_engloss_rate(eleceng, xe, rs)
    
    deposited_heat_vec = np.zeros_like(eleceng)
    
    
    #############################################
    # Initialization of secondary spectra 
    #############################################

    # Low and high energy boundaries
    loweng = 3000
    eleceng_high = eleceng[eleceng > loweng]
    eleceng_high_ind = np.arange(eleceng.size)[eleceng > loweng]
    eleceng_low = eleceng[eleceng <= loweng]
    eleceng_low_ind  = np.arange(eleceng.size)[eleceng <= loweng]


    if eleceng_low.size == 0:
        raise TypeError('Energy abscissa must contain a low energy bin below 3 keV.')

    # Empty containers for quantities.
    # Final secondary photon spectrum.
    sec_phot_tf = tf.TransFuncAtRedshift(
        np.zeros((N_eleceng, N_photeng)), in_eng = eleceng,
        rs = rs*np.ones_like(eleceng), eng = photeng,
        dlnz = -1, spec_type = 'N'
    )
    # Final secondary low energy electron spectrum.
    sec_lowengelec_tf = tf.TransFuncAtRedshift(
        np.zeros((N_eleceng, N_eleceng)), in_eng = eleceng,
        rs = rs*np.ones_like(eleceng), eng = eleceng,
        dlnz = -1, spec_type = 'N'
    )

    # Start building sec_phot_tf and sec_lowengelec_tf.
    # Low energy regime first.

    sec_lowengelec_tf._grid_vals[:eleceng_low.size, :eleceng_low.size] = (
        np.identity(eleceng_low.size)
    )

    # Continuum energy loss rate per electron, dU_CMB/dt.
    CMB_upscatter_eng_rate = phys.thomson_xsec*phys.c*phys.CMB_eng_density(T)
    print('Thomson cross section: ', phys.thomson_xsec)
    print('CMB energy density: ', phys.CMB_eng_density(T))

    # High energy electron loop to get fully resolved spectrum.
    for i, eng in zip(eleceng_high_ind, eleceng_high):
        

        # print('Check energies and indexing: ')
        # print(i, eleceng[i], eng)

        phot_ICS_N = phot_ICS_tf.grid_vals[i]
        
        elec_ICS_N      = elec_ICS_tf.grid_vals[i]
        
        elec_exc_HI_N   = elec_exc_HI_tf.grid_vals[i]
        elec_exc_HeI_N  = elec_exc_HeI_tf.grid_vals[i]
        elec_exc_HeII_N = elec_exc_HeII_tf.grid_vals[i]
        
        elec_ion_HI_N   = elec_ion_HI_tf.grid_vals[i]
        elec_ion_HeI_N  = elec_ion_HeI_tf.grid_vals[i]
        elec_ion_HeII_N = elec_ion_HeII_tf.grid_vals[i]
        
#         if i == 137:
#             utils.compare_arr([eleceng, elec_ion_HI_N])
#             print('Total Number: ', np.sum(elec_ion_HI_N))
#             print('Total Energy: ', np.dot(eleceng, elec_ion_HI_N))
        
        elec_heat_spec = spectools.rebin_N_arr(np.array([1]), np.array([eng]), eleceng)
        elec_heat_spec.eng -= dE_heat_dt[i]
        elec_heat_spec.rebin(eleceng)
        elec_heat_N = elec_heat_spec.N
        
        sec_elec_spec_N = (
            elec_ICS_N 
            + elec_exc_HI_N + elec_exc_HeI_N + elec_exc_HeII_N
            + elec_ion_HI_N + elec_ion_HeI_N + elec_ion_HeII_N
            + elec_heat_N
        )
        sec_phot_spec_N = phot_ICS_N
        

        sec_elec_totN = np.sum(sec_elec_spec_N)
        # The *net* total energy of secondary electrons produced
        # per unit time.
        sec_elec_toteng = np.dot(sec_elec_spec_N, eleceng)
        # The total energy of secondary photons produced per unit time.
        sec_phot_toteng = np.dot(sec_phot_spec_N, photeng)
        # Deposited ICS energy per unit time, dD/dt.
        # Numerical error (should be zero except for numerics)
        print(np.sum(phot_ICS_N))
        print(np.sum(elec_ICS_N))
        deposited_ICS_eng = (
            np.sum(elec_ICS_N)*eng - np.dot(elec_ICS_N, eleceng)
            - (np.dot(phot_ICS_N, photeng) - CMB_upscatter_eng_rate)
        )
        # Deposited excitation energy. 
        deposited_exc_eng = (
            phys.lya_eng*np.sum(elec_exc_HI_N)
            + phys.He_exc_eng*np.sum(elec_exc_HeI_N)
            + 4*phys.lya_eng*np.sum(elec_exc_HeII_N)
        )
        # Deposited ionization energy. Remember that the secondary spectrum
        # has two electrons for each ionization event.
        deposited_ion_eng = (
            phys.rydberg*np.sum(elec_ion_HI_N/2)
            + phys.He_ion_eng*np.sum(elec_ion_HeI_N/2)
            + 4*phys.rydberg*np.sum(elec_ion_HeII_N/2)
        )
        # Deposited heating energy. 
        deposited_heat_eng = dE_heat_dt[i]
        
        # In the original code, the energy of the electron has gamma > 20,
        # then the continuum energy loss is assigned to deposited_eng instead.
        # I'm not sure if this is necessary, but let's be consistent with the
        # original code for now.

        continuum_engloss = CMB_upscatter_eng_rate
        
        if eng + phys.me > 20*phys.me:
            deposited_ICS_eng -= CMB_upscatter_eng_rate
            continuum_engloss = 0

        # Remove self-scattering.
        sec_elec_spec_N[i] = 0
        
        # Rescale.
        toteng_no_self_scatter = (
            np.dot(sec_elec_spec_N, eleceng)
            + np.dot(sec_phot_spec_N, photeng)
            - continuum_engloss
            + deposited_ICS_eng
            + deposited_exc_eng
            + deposited_ion_eng
            + deposited_heat_eng
        )
        
        fac = eng/toteng_no_self_scatter
        # Normalize to one electron. 
        
        sec_elec_spec_N    *= fac
        sec_phot_spec_N    *= fac
        continuum_engloss  *= fac
        deposited_ICS_eng  *= fac
        deposited_exc_eng  *= fac
        deposited_ion_eng  *= fac
        deposited_heat_eng *= fac

        # Get the full secondary photon spectrum. Type 'N'
        resolved_phot_spec_vals = np.dot(
            sec_elec_spec_N, sec_phot_tf.grid_vals
        )
        
        # Get the full secondary low energy electron spectrum. Type 'N'.

        resolved_lowengelec_spec_vals = np.dot(
            sec_elec_spec_N, sec_lowengelec_tf.grid_vals
        )

#         utils.compare_arr([eleceng, resolved_lowengelec_spec_vals])
        
        # The resolved lowengelec spectrum is simply one electron
        # in the bin just below 3 keV.
        # Added directly to sec_lowengelec_tf. Removed the dot for speed.
        # resolved_lowengelec_spec_vals = np.zeros_like(eleceng)
        # resolved_lowengelec_spec_vals[eleceng_low_ind[-1]] += 1

        # Add the resolved spectrum to the first scatter.
        sec_phot_spec_N += resolved_phot_spec_vals

        # Resolve the secondary electron continuum loss and deposition.
        continuum_engloss += np.dot(sec_elec_spec_N, cont_loss_ICS_vec)

        deposited_ICS_eng  += np.dot(sec_elec_spec_N, deposited_ICS_vec)
        deposited_exc_eng  += np.dot(sec_elec_spec_N, deposited_exc_vec)
        deposited_ion_eng  += np.dot(sec_elec_spec_N, deposited_ion_vec)
        deposited_heat_eng += np.dot(sec_elec_spec_N, deposited_heat_vec)

        # Now, append the resulting spectrum to the transfer function.
        # Do this without calling append of course: just add to the zeros
        # that fill the current row in _grid_vals.
        sec_phot_tf._grid_vals[i] += sec_phot_spec_N
        sec_lowengelec_tf._grid_vals[i] += resolved_lowengelec_spec_vals
        # Set the correct values in cont_loss_vec and deposited_vec.
        
        cont_loss_ICS_vec[i]  = continuum_engloss
        deposited_ICS_vec[i]  = deposited_ICS_eng
        deposited_exc_vec[i]  = deposited_exc_eng
        deposited_ion_vec[i]  = deposited_ion_eng
        deposited_heat_vec[i] = deposited_heat_eng

        
        check = True
        verbose = True
        failed_conservation_check = False
        
        if check:

            conservation_check = (
                eng
                - np.dot(sec_lowengelec_tf.grid_vals[i], eleceng)
                + cont_loss_ICS_vec[i]
                - np.dot(sec_phot_tf.grid_vals[i], photeng)
                - deposited_exc_vec[i]
                - deposited_ion_vec[i]
                - deposited_heat_vec[i]
            )

            if conservation_check/eng > 0.1:
                failed_conservation_check = True
                
            if verbose or failed_conservation_check:
                
                print('***************************************************')
                print('rs: ', rs)
                print('injected energy: ', eng)
                print(
                    'Energy in low energy electrons: ',
                    np.dot(sec_lowengelec_tf.grid_vals[i], eleceng)
                )
                print('Energy in photons: ', np.dot(sec_phot_tf.grid_vals[i], photeng))
                print('Continuum_engloss: ', cont_loss_ICS_vec[i])
                print(
                    'Energy in photons - Continuum: ',
                    np.dot(sec_phot_tf.grid_vals[i], photeng) - cont_loss_ICS_vec[i]
                )
                print(
                    'Deposited in ionization: ', deposited_ion_vec[i]
                )
                print(
                    'Deposited in excitation: ', deposited_exc_vec[i]
                )
                print(
                    'Deposited in heating: ', deposited_heat_vec[i]
                )
                print(
                    'Energy is conserved up to (%): ',
                    conservation_check/eng*100
                )
                print('Deposited in ICS: ', deposited_ICS_vec[i])
                print(
                    'Energy conservation with deposited (%): ',
                    (conservation_check - deposited_ICS_vec[i])/eng*100
                )
                print('***************************************************')
                
            if failed_conservation_check:
                raise RuntimeError('Conservation of energy failed.')


    return (sec_phot_tf, sec_lowengelec_tf, cont_loss_ICS_vec)


In [254]:
eleceng = 10**np.arange(0, np.log10(5e12), 0.0254)
photeng = 10**np.arange(-4, np.log10(5e12), 0.0334)


rs = 600
xe = .01

b = get_elec_cooling_tf_fast(
    ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf,
    eleceng, photeng, rs, xe, xHe=0
)

Thomson cross section:  6.652458734e-25
CMB energy density:  33769934343.76002
0.0017693426250652959
0.00018868430148974896
***************************************************
rs:  600
injected energy:  3018.5613014197993
Energy in low energy electrons:  2901.852478927338
Energy in photons:  2689.608166251477
Continuum_engloss:  2647.716839442689
Energy in photons - Continuum:  41.89132680878811
Deposited in ionization:  40.38844151962926
Deposited in excitation:  28.02771738581055
Deposited in heating:  6.460473298242648
Energy is conserved up to (%):  -0.0019590962085588894
Deposited in ICS:  -0.059136520008484246
Energy conservation with deposited (%):  -2.1762421206016872e-14
***************************************************
0.0017693419375264348
0.00019423000239059208
***************************************************
rs:  600
injected energy:  3200.368594586517
Energy in low energy electrons:  2964.2361521465746
Energy in photons:  5421.598177182115
Continuum_engloss:  5334.52

0.0017693416692647675
0.0006309288680159583
***************************************************
rs:  600
injected energy:  37325.015779572066
Energy in low energy electrons:  7568.548454798597
Energy in photons:  304837.49279381055
Continuum_engloss:  279614.6733238883
Energy in photons - Continuum:  25222.81946992228
Deposited in ionization:  2402.291521670988
Deposited in excitation:  1776.5682370029124
Deposited in heating:  360.8937150935893
Energy is conserved up to (%):  -0.016357980803914287
Deposited in ICS:  -6.105618916297932
Energy conservation with deposited (%):  4.7032388597088606e-14
***************************************************
0.001769341669116983
0.0006476044524893163
***************************************************
rs:  600
injected energy:  39573.09339293611
Energy in low energy electrons:  7674.263952336832
Energy in photons:  316527.6393042116
Continuum_engloss:  289260.26001551095
Energy in photons - Continuum:  27267.379288700642
Deposited in ionization

***************************************************
rs:  600
injected energy:  120226.4434617413
Energy in low energy electrons:  9244.300053886454
Energy in photons:  580859.1189653104
Continuum_engloss:  475970.2650195386
Energy in photons - Continuum:  104888.85394577181
Deposited in ionization:  3205.3325443032672
Deposited in excitation:  2411.107512742238
Deposited in heating:  487.2261540884844
Energy is conserved up to (%):  -0.008631003922421323
Deposited in ICS:  -10.376749050706508
Energy conservation with deposited (%):  -2.196169769360896e-13
***************************************************
0.0017693417036913981
0.001039473472733486
***************************************************
rs:  600
injected energy:  127467.6561025312
Energy in low energy electrons:  9303.602959809728
Energy in photons:  597624.9699616223
Continuum_engloss:  485610.10047426214
Energy in photons - Continuum:  112014.86948736012
Deposited in ionization:  3233.655136761497
Deposited in excitation

0.0017693417849957566
0.001609430609821722
***************************************************
rs:  600
injected energy:  828323.5329759169
Energy in low energy electrons:  10261.231114164031
Energy in photons:  1555354.2243176769
Continuum_engloss:  744374.0813372995
Energy in photons - Continuum:  810980.1429803773
Deposited in ionization:  3690.4398847352945
Deposited in excitation:  2809.1747300494885
Deposited in heating:  598.7809498746412
Energy is conserved up to (%):  -0.001960186163674982
Deposited in ICS:  -16.236683283159397
Energy conservation with deposited (%):  -8.430324738304101e-14
***************************************************
0.0017693417853869627
0.0016200063092564746
***************************************************
rs:  600
injected energy:  878213.2798980062
Energy in low energy electrons:  10270.700918969149
Energy in photons:  1611100.6462831316
Continuum_engloss:  750250.4525977722
Energy in photons - Continuum:  860850.1936853593
Deposited in ionizati

0.0017693418150264964
0.0017455334012232654
***************************************************
rs:  600
injected energy:  3179801.4276915244
Energy in low energy electrons:  10362.522826660284
Energy in photons:  4002896.7081547985
Continuum_engloss:  840659.7021526906
Energy in photons - Continuum:  3162237.0060021076
Deposited in ionization:  3738.705109632618
Deposited in excitation:  2850.327331229893
Deposited in heating:  631.2048923488104
Energy is conserved up to (%):  -0.0005767174734800311
Deposited in ICS:  -18.338470456338214
Energy conservation with deposited (%):  2.7476481743063503e-14
***************************************************
0.0017693418186372095
0.0017477341272808355
***************************************************
rs:  600
injected energy:  3371320.1787284347
Energy in low energy electrons:  10363.760724290505
Energy in photons:  4196947.836900086
Continuum_engloss:  843195.2078846614
Energy in photons - Continuum:  3353752.6290154248
Deposited in ioniz

...

0.0017378205991212862
0.0017378205991212857
***************************************************
rs:  600
injected energy:  4757732275.412052
Energy in low energy electrons:  10375.080539028007
Energy in photons:  4758587719.957582
Continuum_engloss:  873049.2124453796
Energy in photons - Continuum:  4757714670.745137
Deposited in ionization:  3744.6885759586526
Deposited in excitation:  2855.561413830182
Deposited in heating:  648.3849746593552
Energy is conserved up to (%):  -4.0037116750868524e-07
Deposited in ICS:  -19.04858875680194
Energy conservation with deposited (%):  1.0483926833294657e-14
***************************************************
0.0017359929166242208
0.00173599291662422
***************************************************
rs:  600
injected energy:  5044289459.52414
Energy in low energy electrons:  10375.080540135403
Energy in photons:  5045144904.068779
Continuum_engloss:  873049.2124481974
Energy in photons - Continuum:  5044271854.856331
Deposited in ionization: 

***************************************************
rs:  600
injected energy:  27504262230.30312
Energy in low energy electrons:  10375.080549244258
Energy in photons:  27505117674.83459
Continuum_engloss:  873049.2124713906
Energy in photons - Continuum:  27504244625.62212
Deposited in ionization:  3744.6885808302654
Deposited in excitation:  2855.5614184494925
Deposited in heating:  648.3994386151571
Energy is conserved up to (%):  -6.925831759097037e-08
Deposited in ICS:  -19.0489847435793
Energy conservation with deposited (%):  -1.6517212152806578e-14
***************************************************
0.0016050572077762459
0.001605057207776246
***************************************************
rs:  600
injected energy:  29160837985.21234
Energy in low energy electrons:  10375.080549286338
Energy in photons:  29161693429.743587
Continuum_engloss:  873049.2124714988
Energy in photons - Continuum:  29160820380.531116
Deposited in ionization:  3744.688580850331
Deposited in excitati

0.0013179450427482984
0.0013179450427482984
***************************************************
rs:  600
injected energy:  118686138464.49168
Energy in low energy electrons:  10375.080549639164
Energy in photons:  118686993909.02014
Continuum_engloss:  873049.2124723995
Energy in photons - Continuum:  118686120859.80766
Deposited in ionization:  3744.688581018584
Deposited in excitation:  2855.5614186303146
Deposited in heating:  648.402686317276
Energy is conserved up to (%):  -1.6050076583493326e-08
Deposited in ICS:  -19.04920726067639
Energy conservation with deposited (%):  -7.462426169339481e-15
***************************************************
0.0013020449767743505
0.0013020449767743502
***************************************************
rs:  600
injected energy:  125834578869.02853
Energy in low energy electrons:  10375.080549642487
Energy in photons:  125835434313.557
Continuum_engloss:  873049.212472408
Energy in photons - Continuum:  125834561264.34453
Deposited in ionizat

0.0008056806198739039
0.0008056806198739039
***************************************************
rs:  600
injected energy:  647142615748.5831
Energy in low energy electrons:  10375.080549673397
Energy in photons:  647143471193.1112
Continuum_engloss:  873049.2124724868
Energy in photons - Continuum:  647142598143.8987
Deposited in ionization:  3744.6885810349095
Deposited in excitation:  2855.5614186461626
Deposited in heating:  648.4043951940913
Energy is conserved up to (%):  -2.9437889849348187e-09
Deposited in ICS:  -19.050519033316238
Energy conservation with deposited (%):  9.262394786543807e-16
***************************************************
0.0007880799558765857
0.0007880799558765858
***************************************************
rs:  600
injected energy:  686119875281.7047
Energy in low energy electrons:  10375.080549673616
Energy in photons:  686120730726.2323
Continuum_engloss:  873049.2124724875
Energy in photons - Continuum:  686119857677.0198
Deposited in ionizati

***************************************************
rs:  600
injected energy:  2484276893696.798
Energy in low energy electrons:  10375.080549675416
Energy in photons:  2484277749141.325
Continuum_engloss:  873049.2124724919
Energy in photons - Continuum:  2484276876092.113
Deposited in ionization:  3744.688581035873
Deposited in excitation:  2855.5614186471144
Deposited in heating:  648.4055368135557
Energy is conserved up to (%):  -7.668647708868422e-10
Deposited in ICS:  -19.051483335389708
Energy conservation with deposited (%):  1.7672198622997363e-14
***************************************************
0.0004324791376659039
0.00043247913766590384
***************************************************
rs:  600
injected energy:  2633904352747.313
Energy in low energy electrons:  10375.080549675378
Energy in photons:  2633905208191.841
Continuum_engloss:  873049.2124724919
Energy in photons - Continuum:  2633904335142.6284
Deposited in ionization:  3744.6885810358535
Deposited in excita